# Into to GRiTS

In [1]:
import mbuild as mb
import numpy as np

from grits import backmap
from grits import CG_Compound

### Coarse-graining

A `CG_Compound` is created using an `mbuild.Compound` and a dictionary of the form `{"bead name": "bead SMARTS"}`.

[SMILES](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html)/[SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) strings are ways of specifying chemical information in a string format. GRiTS uses [openbabel](http://openbabel.org/docs/current/UseTheLibrary/Python_Pybel.html) for SMARTS matching and [RDKit](https://rdkit.readthedocs.io/en/latest/) for loading SMILES representations.

The `CG_Compound` class is built on the `mbuild.Compound` class (more about [mBuild](https://mbuild.mosdef.org/en/stable/)) but has some extra feautures which make it convenient for coarse grain structures. 

First we'll load a [3-hexylthiophene polymer (P3HT)](https://en.wikipedia.org/wiki/Polythiophene) into an `mbuild.Compound` and define a color dictionary. (The color definitions are unnecessary but helpful for distinguishing the coarse-grained beads during visualization.)

In [2]:
p3ht = mb.load("../grits/tests/assets/P3HT_16.mol2")

p3ht_colors = {"_B": "blue", "_S": "orange"}

In the example below, the `cg_beads` dictionary specifies a thiophene ring to be mapped to a `"_B"` bead (B for backbone!) with the string `"c1sccc1"` and three alkyl carbons to be mapped to an `"_S"` bead (S for sidechain!) with the string `"CCC"`.

And that's all you need to create a `CG_Compound`! All the information for bonding and backmapping is inferred during initialization.

In [3]:
cg_beads = {"_B": "c1sccc1", "_S": "CCC"}

cg_p3ht = CG_Compound(p3ht, cg_beads)

cg_p3ht.visualize(color_scheme=p3ht_colors, show_atomistic=True).show()
cg_p3ht.visualize(color_scheme=p3ht_colors).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Backmapping

Finally, after using a `CG_compound` model for a simulation, we may want to retrieve the atomistic structure. For this we can use the `backmap` function. A `CG_Compound` keeps track of the bonding and anchors during initialization and uses these to recreate the  atomistic positions and bonds.

In [4]:
fine_grained = backmap(cg_p3ht)
fine_grained.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol